In [1]:
import pandas as pd
import numpy as np

In [2]:
X_train_in = pd.read_csv("X_train.csv", index_col=0)
y_train_in = pd.read_csv("y_train.csv",index_col=0)
X_val = pd.read_csv("X_test.csv", index_col=0)
info = pd.read_csv("studs_info.csv").drop_duplicates(subset=["STD_ID"])

In [3]:
X_train_in = X_train_in.reset_index().drop(columns=["index"])
X_train_in = pd.merge(X_train_in.reset_index(), info).sort_values(by="index")

X_val = X_val.reset_index().drop(columns=["index"])
X_val = pd.merge(X_val.reset_index(), info).sort_values(by="index")

X_train_in.index = pd.read_csv("X_train.csv", index_col=0).index
X_val.index = pd.read_csv("X_test.csv", index_col=0).index
X_val = X_val.drop(columns=["index"])
X_train_in = X_train_in.drop(columns=["index"])

In [4]:
X_train_in['НАПРАВЛЕНИЕ_clustered'] = X_train_in['НАПРАВЛЕНИЕ'].agg([lambda x: x[:2]])
X_val['НАПРАВЛЕНИЕ_clustered'] = X_val['НАПРАВЛЕНИЕ'].agg([lambda x: x[:2]])

In [5]:
att_types = list(set(list(X_train_in["АТТЕСТАЦИЯ"].values) + list(X_val["АТТЕСТАЦИЯ"].values)))
disc_types = list(set(list(X_train_in["ДИСЦИПЛИНА"].values) + list(X_val["ДИСЦИПЛИНА"].values)))
napr_types = list(set(list(X_train_in["НАПРАВЛЕНИЕ"].values) + list(X_val["НАПРАВЛЕНИЕ"].values)))
napr_c_types = list(set(list(X_train_in["НАПРАВЛЕНИЕ_clustered"].values) + list(X_val["НАПРАВЛЕНИЕ_clustered"].values)))
school = list(set(list(X_train_in["Что именно закончил"].values) + list(X_val["Что именно закончил"].values)))

In [6]:
lyc = ["лицей"]
sch = ["сош", "школа", "осш", "ош"]
gym = ['гимназия']
tech = ["техникум"]
col = ["колледж"]
cor = ["коррекционная"]
uni = ["университет", "академия"]
uch = ["училище", "пу", "пту"]

In [7]:
lycs=[]
schs=[]
gyms=[]
techs=[]
cols=[]
cors=[]
unis=[]
uchs=[]
for i in school:
    for j in lyc:
        if j in str(i).lower():
            lycs.append(i)
    
    for j in sch:
        if j in str(i).lower() and "коррекционная" not in str(i).lower():
            schs.append(i)
    
    for j in gym:
        if j in str(i).lower():
            gyms.append(i)
    
    for j in tech:
        if j in str(i).lower():
            techs.append(i)
    
    for j in col:
        if j in str(i).lower():
            cols.append(i)
    
    for j in cor:
        if j in str(i).lower():
            cors.append(i)
    
    for j in uni:
        if j in str(i).lower():
            unis.append(i)
    
    for j in uch:
        if j in str(i).lower():
            uchs.append(i)
lycs,schs,gyms,techs,cols,cors,unis,uchs

(['ПРОФЕССИОНАЛЬНЫЙ ЛИЦЕЙ 20 Г.СТАРАЯ РУССА',
  'Государственный профессиональный лицей №3',
  'Профессиональный лицей № 17, ВЕЛИКИЙ НОВГОРОД',
  'ГУНПО технологический профессиональный лицей',
  'ГОУ НПО "Агролицей №25" , СТАРАЯ РУССА',
  'Профессиональный лицей № 1, ВЕЛИКИЙ НОВГОРОД',
  'ЛИЦЕЙ-ИНТЕРНАТ , НОВГОРОД'],
 ['СРЕДНЯЯ ШКОЛА N   4, ЧУДОВО',
  'Вечерняя школа № 1, г.НОВГОРОД',
  'МСОШ  Уторгош ,ШИМСКИЙ р-н',
  'МСОШ  Уторгош ,ШИМСКИЙ р-н',
  'МОУ СОШ д.Мойка',
  'МОУ СОШ д.Мойка',
  'МОУ СОШ N3, СОЛЬЦЫ',
  'МОУ СОШ N3, СОЛЬЦЫ',
  'МСОШ N   8 с углубл. изучением математики, СТАРАЯ РУССА',
  'МСОШ N   8 с углубл. изучением математики, СТАРАЯ РУССА',
  'МОУ "Средняя общеобразовательная школа № 20 им.Кирилла и Мефодия"',
  'МСОШ (полная) N 4, МАЛАЯ ВИШЕРА',
  'МСОШ (полная) N 4, МАЛАЯ ВИШЕРА',
  'МСОШ N   2, ХВОЙНАЯ',
  'МСОШ N   2, ХВОЙНАЯ',
  'МОУ СОШ (полная), ХОЛМ',
  'МОУ СОШ (полная), ХОЛМ',
  'СРЕДНЯЯ ШКОЛА № 36 ВЕЛИКИЙ НОВГОРОД',
  'МСОШ (полная)  N   1, МАЛАЯ ВИШЕРА',
  '

In [8]:
X_train_in["mark"]=y_train_in["mark"]

In [9]:
def track(x):
    d = [lycs,schs,gyms,techs,cols,cors,unis,uchs]
    a = ["Лицей", "Школа", "Гимназия", "Техникум", "Колледж", "Коррекция", "Университет", "Училище"]
    for i in range(len(d)):
        if x in d[i]:
            return a[i]
    return "Школа"

In [10]:
X_train_in["Что именно закончил"] = X_train_in["Что именно закончил"].agg([track])
X_val["Что именно закончил"] = X_val["Что именно закончил"].agg([track])

In [11]:
X_val

,STD_ID,НАПРАВЛЕНИЕ,ГОД,КУРС,СЕМЕСТР,АТТЕСТАЦИЯ,ДИСЦИПЛИНА,number,Пол,Статус,Дата выпуска,Категория обучения,Форма обучения,Шифр,направление (специальность),,Образование,Дата выдачи,Что именно закончил,НАПРАВЛЕНИЕ_clustered
56065,163261,35.03.04 Агрономия,2018-2019,3.0,5.0,Дифференцированный зачет,История,594.0,Ж,СТ,NaN,БП,З,35.03.04,Агрономия,4972.0,СП,2016-06-24 00:00:00,Колледж,35
304637,162990,23.03.03 Эксплуатация транспортно-технологичес...,2019-2020,4.0,7.0,Экзамен,Сертификация и лицензирование в сфере производ...,3079.0,Ж,СТ,NaN,БП,Д,23.03.03,Эксплуатация транспортно-технологических машин...,5027.0,СР,2016-06-23 00:00:00,Школа,23
177437,192595,40.03.01 Юриспруденция,2018-2019,1.0,2.0,Экзамен,История государства и права России,6663.0,М,СТ,NaN,ДОГ,Д,40.03.01,Юриспруденция,6998.0,СР,2018-06-27 00:00:00,Школа,40
222396,198675,38.03.01 Экономика,2018-2019,2.0,3.0,Дифференцированный зачет,Иностранный язык в сфере профессиональной комм...,3911.0,М,СТ,NaN,ДОГ,Д,38.03.01,Экономика,5525.0,СР,2017-06-23 00:00:00,Школа,38
271996,162111,44.03.05 Педагогическое образование (с двумя п...,2018-2019,3.0,6.0,Экзамен,Физическая химия,3460.0,Ж,СТ,NaN,БП,Д,44.03.05,Педагогическое образование (с двумя профилями ...,5102.0,СР,2016-06-24 00:00:00,Школа,44
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49838,164804,31.05.01 Лечебное дело,2019-2020,4.0,7.0,Экзамен,"Факультетская хирургия, урология",3182.0,Ж,СТ,NaN,БП,Д,31.05.01,Лечебное дело,5069.0,СР,2016-06-23 00:00:00,Школа,31
273753,170508,40.03.01 Юриспруденция,2019-2020,4.0,7.0,Дифференцированный зачет,Арбитражный процесс,462.0,Ж,СТ,NaN,ДОГ,З,40.03.01,Юриспруденция,1075.0,ОО,2011-06-15 00:00:00,Школа,40
35426,178316,31.05.03 Стоматология,2017-2018,1.0,2.0,Экзамен,"Гистология, эмбриология, цитология - гистологи...",5088.0,Ж,СТ,NaN,ДОГ,Д,31.05.03,Стоматология,6285.0,СР,2017-06-28 00:00:00,Школа,31
160476,186542,46.03.01 История,2019-2020,2.0,3.0,Дифференцированный зачет,Иностранный язык,1095.0,Ж,СТ,NaN,БП,Д,46.03.01,История,6423.0,СП,2018-06-26 00:00:00,Колледж,46


In [12]:
def add_feat_mean(cat):
    mean_mark = X_train_in.groupby(cat).mean()["mark"]
    X_train_in.loc[:,f"{cat}_mean"] = X_train_in.loc[:,cat].map(mean_mark)
    X_val.loc[:,f"{cat}_mean"] = X_val.loc[:,cat].map(mean_mark)
    
add_feat_mean("STD_ID")
add_feat_mean("ДИСЦИПЛИНА")
add_feat_mean("АТТЕСТАЦИЯ")
add_feat_mean("НАПРАВЛЕНИЕ")
add_feat_mean("НАПРАВЛЕНИЕ_clustered")
add_feat_mean("ГОД")
add_feat_mean("КУРС")
add_feat_mean("СЕМЕСТР")
add_feat_mean("Категория обучения")
add_feat_mean("Пол")
add_feat_mean("Что именно закончил")

In [13]:
y_train_in["mark"]

index
51081     5
72921     5
96438     5
273044    4
371309    3
         ..
224856    3
260998    3
57245     4
218529    4
233318    4
Name: mark, Length: 105058, dtype: int64

In [14]:
X_train_in.to_csv("X_train_app.csv")
X_val.to_csv("X_val_app.csv")